# Test the performance of the fine-tunning model

In [44]:
# import all the model we need
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from transformers import BertTokenizerFast, BertForSequenceClassification
import pandas as pd
import numpy as np

In [27]:
# load the fine-tunning model
model_name = "bert-base-uncased"
model = BertForSequenceClassification.from_pretrained("./saved/")
tokenizer = BertTokenizerFast.from_pretrained(model_name)


In [36]:
# choose 8000 comments form the training dataset for testing
test_num = 8000
df = pd.read_csv("train.csv")
data = np.array(df)
data = data[:,1:]
selected_indices = np.random.choice(data.shape[0], test_num, replace=False)
data_test = data[selected_indices, :]
test_texts = data_test[:, 0]
test_labels = data_test[:, 1:]
print(test_labels.shape)
print(test_texts.shape)

(8000, 6)
(8000,)


In [38]:
# the threshold is 0.5, if the predicted probability>0.5, the label will be 1
# mainly test the fine-tunning model's performance by calculating the accuracy and F1 score of the 6 labels
threshold = 0.5
acc = np.zeros(6)
TP = np.zeros(6)
FP = np.zeros(6)
FN = np.zeros(6)
for i in range(test_num):
    text = test_texts[i]
    y = test_labels[i]
    input_val = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors="pt")
    output_val = model(**input_val)
    probabilities = torch.sigmoid(output_val.logits)
    temp = np.array(probabilities.tolist()[0])
    temp = np.where(temp > threshold, 1, 0)
    for j in range(6):
        if temp[j]==y[j]:
            acc[j] += 1
        if y[j]==1 and temp[j]==1:
            TP[j] += 1
        if y[j]==0 and temp[j]==1:
            FP[j] += 1
        if y[j]==1 and temp[j]==0:
            FN[j] += 1
    if i % 100 == 0:
        print(i)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900


In [39]:
print(acc)
print(TP)
print(FP)
print(FN)

[7742. 7932. 7906. 7977. 7831. 7930.]
[549.  29. 365.   1. 312.   1.]
[49.  7. 39.  1. 85.  2.]
[209.  61.  55.  22.  84.  68.]


In [40]:
P = TP/(TP+FP)
R = TP/(TP+FN)
print(P)
print(R)

[0.9180602  0.80555556 0.90346535 0.5        0.78589421 0.33333333]
[0.72427441 0.32222222 0.86904762 0.04347826 0.78787879 0.01449275]


In [42]:
acc = acc/test_num
F_1 = 2*(P*R)/(P+R)
print(acc, F_1)

[0.96775  0.9915   0.98825  0.997125 0.978875 0.99125 ] [0.80973451 0.46031746 0.88592233 0.08       0.78688525 0.02777778]


In [43]:
print(np.mean(acc))
print(np.mean(F_1))

0.9857916666666666
0.5084395545613835


Overall, the fine-tuned model has a good performance. 

The accuracy of every label, toxic, severe toxic, obscene, insult, identity hate, are 0.96775, 0.9915, 0.98825, 0.997125, 0.978875, 0.99125<br>
The average accuracy is 0.9858.

The model seems to perform well in terms of overall accuracy. However, accuracy may not always be a reliable metric for imbalanced datasets. Therefore, it is essential to analyze the F1 scores as well.

The F1 of every label, toxic, severe toxic, obscene, insult, identity hate, are 0.80973451, 0.46031746, 0.88592233, 0.08, 0.78688525, 0.02777778<br>
The average F1 score is 0.5084.
- The F1 scores for 'toxic' (0.8097), 'obscene' (0.8859), and 'identity_hate' (0.7869) labels are relatively high, which indicates good performance in these categories.
- The F1 scores for 'severe_toxic' (0.4603) and 'insult' (0.0800) labels are relatively low, which indicates the model is not performing well in these categories. The low F1 score for 'severe_toxic' could be due to the imbalanced nature of the dataset or insufficient training data for this category. For the 'insult' category, the low F1 score could be a result of difficulty in distinguishing insults from other types of toxic content.
- The F1 score for 'threat' (0.0278) is extremely low, which indicates the model performs poorly in this category. This may be due to the rarity of threat instances in the dataset or a high number of false positives and false negatives.

Overall, the fine-tuned model appears to have a good performance in some categories ('toxic', 'obscene', and 'identity_hate') but struggles with others ('severe_toxic', 'insult', and 'threat'). 